## Train and load

In [26]:
import os
import pandas as pd
import xgboost as xgb
from pickle import load, dump
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
import joblib 
from xgboost.training import train
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

filterwarnings("ignore")


data_folder = "data"
features_csv = os.path.join(data_folder, "features.csv")
targets_csv = os.path.join(data_folder, "targets.csv")


features = pd.read_csv(features_csv, index_col=0)
targets = pd.read_csv(targets_csv, index_col=0)


features.drop(['index','season'],axis=1,inplace=True)

#le= OneHotEncoder(sparse=False)
le= LabelEncoder()
targets=le.fit_transform(targets)
joblib.dump(le,os.path.join('models','label_encoder.joblib'))
print(features.shape)
print(targets.shape)

train_X, test_X, train_Y, test_Y = train_test_split(features, targets, test_size=0.3)

(10044, 122)
(10044,)


In [27]:

dtrain = xgb.DMatrix(train_X, label=train_Y, enable_categorical=True)
dtest = xgb.DMatrix(test_X, label=test_Y, enable_categorical=True)
# dtrain = xgb.DMatrix(train_X, label=train_Y)
# dtest = xgb.DMatrix(test_X, label=test_Y)


param = {
        "verbosity": 0,
        "objective": "multi:softprob",
        "eval_metric": "auc",
        'num_class': 3,
        "booster": 'gblinear',
        "lambda": 0.996390119803819,
        "alpha": 1.510122265371948e-08,
    }

num_round = 20  # the number of training iterations

xgbc=xgb.train(param,dtrain,num_round)

# Evaluation

In [28]:
xgbc.eval(dtest)

'[0]\teval-auc:0.602694'

In [33]:
y_preds=xgbc.predict(dtest)

* predictions shape n_samples*n_classes. we should encode the true values as well

In [34]:
ohe=OneHotEncoder(sparse=False)
test_Y_ohe=ohe.fit_transform(test_Y.reshape(-1, 1))

from sklearn.metrics import roc_auc_score
roc_auc_score(test_Y_ohe,y_preds,average='weighted')

0.6026940012398101

In [100]:
model_file_path = os.path.join("models", "xgb_model.json")

xgbc.save_model(model_file_path)

In [102]:
from sklearn.metrics import classification_report, roc_auc_score, f1_score

In [105]:
y_preds

XGBoostError: [02:04:56] ..\src\metric\rank_metric.cc:243: Check failed: preds.Size() == info.labels_.Size() (9042 vs. 3014) : label size predict size not match